# Transformación y Carga

### Importación de librerías

Del paquete "connection_db", ubicado en el directorio "source del directorio", se importa el módulo "db_utils" que proporciona una funciones centralizada para establecer y gestionar conexiones a una base de datos PostgreSQL. Su objetivo es promover la reutilización de código, la modularidad y la consistencia en el acceso a la base de datos dentro de los diferentes notebooks de este proyecto.

In [3]:
import sys
import os
import pandas as pd
import numpy as np
from psycopg2 import sql
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath('../source'))
from connection_db.db_utils import get_connection , close_connection

### Conexión a la base de datos



Este fragmento de código establece una conexión a una base de datos PostgreSQL utilizando la función "get_connection" del módulo "connection_db.db_utils", lee todos los datos de la tabla "accidents" en un DataFrame de Pandas, muestra las primeras filas del DataFrame y luego cierra la conexión a la base de datos utilizando la función "close_connection" del mismo módulo, para liberar recursos.

In [4]:
engine = get_connection()
df = pd.read_sql_query("SELECT * FROM accidents", engine)
df.head()
close_connection(engine)

### Eliminación de columnas redundantes

Este código utiliza la función `unique()` de Pandas para identificar y mostrar los valores únicos presentes en las columnas "country" y "region" del DataFrame `df`. Primero, imprime los valores únicos de la columna "country", seguido de un salto de línea y los valores únicos de la columna "region". Esto permite inspeccionar rápidamente las categorías o valores distintos presentes en estas columnas, lo cual es útil para comprender la distribución de los datos de estás columnas.

In [5]:
print("Valores únicos en la columna 'country':")
print(df["country"].unique())

print("\nValores únicos en la columna 'region':")
print(df["region"].unique())

In [6]:
df = df.drop(columns=["region"])

La columna "region" ha sido eliminada del dataset, ya que no aporta información relevante para los objetivos del proyecto. Dado que ya contamos con la columna "country", la cual especifica el país donde ocurrió cada accidente, la variable "region" se vuelve redundante. Además, el nivel de detalle que proporciona la columna de país es más preciso y útil para el análisis, mientras que la región o continente es una categorización más amplia que no aporta valor significativo a nuestro estudio. 

### Conformidad 

#### Número de cifras decimales

El código examina las columnas con tipo de datos 'float' en DataFrame `df` determinando el número máximo de decimales presentes en cada columna para comprender la precisión de los datos. Luego, redondea todos los valores de estas columnas a dos decimales, simplificando los datos y mejorando su legibilidad para análisis y presentación.

In [7]:
columnas_float = df.select_dtypes(include=['float'])


for col in columnas_float.columns:
    max_decimales = columnas_float[col].astype(str).str.split('.').str[1].str.len().max()
    print(f"Número máximo de decimales en la columna '{col}': {max_decimales}")


In [8]:
columnas_float_redondeadas = columnas_float.round(2)

print(columnas_float_redondeadas.head())

#### Valores de la columna 'driver_alcohol_level'

Los niveles de alcohol en la sangre pueden ser agrupados en categorías significativas (Bajo, Moderado, Alto, etc.) que representan diferentes niveles de riesgo o impacto, en lugar de tratarse como simples valores continuos difíceles de interpretar para el público general. Para esto se utiliza el "Binning", que  implica convertir una variable numérica continua en una variable categórica ordinal al agrupar los valores en intervalos o "bins".

Este código define una función llamada `categorizar_alcohol_level` que clasifica los niveles de alcohol en la sangre (driver_alcohol_level) en categorías como "Bajo", "Moderado", "Alto", "Peligroso" y "Letal", basándose en umbrales específicos. Luego, aplica esta función a la columna "driver_alcohol_level" del DataFrame `df` para crear una nueva columna llamada "Alcohol_Level_Category", que contiene las categorías correspondientes para cada valor de nivel de alcohol. Finalmente, imprime la columna "Alcohol_Level_Category" para mostrar las clasificaciones resultantes. 

In [9]:
def categorizar_alcohol_level(driver_alcohol_level):
    if driver_alcohol_level < 0.03:
        return "Bajo"
    elif driver_alcohol_level < 0.08:
        return "Moderado"
    elif driver_alcohol_level < 0.20:
        return "Alto"
    elif driver_alcohol_level < 0.30:
        return "Peligroso"
    else:
        return "Letal"

df["Alcohol_Level_Category"] = df["driver_alcohol_level"].apply(categorizar_alcohol_level)

print(df["Alcohol_Level_Category"])


Esta conversión no solo optimiza la representación de los datos en visualizaciones, sino que también permite la identificación de patrones clave entre los niveles de alcohol y otros factores críticos, como la severidad de los accidentes o el número de víctimas. 


#### Valores de la columna 'visibility_level'

La función `categorize_visibility` reemplaza la columna "visibility_level" por "Visibility_Category", la cual almacena la clasificación correspondiente para cada registro, pasando de un valor numérico a una categoría descriptiva: Muy Baja, Baja, Moderada o Alta. Esto permite transformar una variable numérica en categórica, facilitando la interpretación de los datos y su representación en visualizaciones.


In [10]:
def categorize_visibility(visibility_level):
    if visibility_level < 200:
        return "Muy Baja"
    elif visibility_level < 300:
        return "Baja"
    elif visibility_level < 400:
        return "Moderada"
    else:
        return "Alta"

df["Visibility_Category"] = df["visibility_level"].apply(categorize_visibility)

print(df["Visibility_Category"])

#### Valores de "days_order", "months_order" y "time_of_day"

In [11]:
print("\nValores únicos en la columna 'time_of_day':")
print(df["time_of_day"].unique())

Las columnas "day_of_week", "month" y "time, of day" son variables categóricas ordinales, lo que significa que las categorías tienen un orden lógico (los días de la semana y los meses del año tienen un orden específico).Al estructurar las variables con un orden definido, se facilita la representación gráfica en gráficos de tendencia o análisis estacionales, evitando errores en la disposición de los datos. Asimismo, esta verificación contribuye a la correcta interpretación de patrones temporales en la ocurrencia de accidentes, lo que puede ser clave para la toma de decisiones en seguridad vial y planificación de estrategias preventivas.

Este código transforma las columnas categóricas "day_of_week", "month" y "time_of_day" en un tipo de dato categórico ordenado en Pandas. Esto permite que el DataFrame conozca el orden lógico de los días de la semana, los meses del año, y el tiempo del día. Se utiliza `pd.Categorical directamente, que es más eficiente y conciso que pd.CategoricalDtype y .astype(). Además, se ha simplificado la impresión de los resultados, eliminando la necesidad de verificar si las columnas están ordenadas, ya que pd.Categorical con ordered=True garantiza que lo estén. Esto facilita el análisis de datos temporales donde el orden es crucial.


In [12]:
days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
months_order = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
time_of_day_order = ["Morning", "Afternoon", "Evening", "Night"]


df["day_of_week"] = pd.Categorical(df["day_of_week"], categories=days_order, ordered=True)
df["month"] = pd.Categorical(df["month"], categories=months_order, ordered=True)
df["time_of_day"] = pd.Categorical(df["time_of_day"], categories=time_of_day_order, ordered=True)

print("Día de semana (Categórico):\n", df["day_of_week"].head(5))
print("\nMes (Categórico):\n", df["month"].head(5))
print("\nTiempo del día (Categórico):\n", df["time_of_day"].head(5))


#### Valores únicos de otras columnas con dtype String

Con el el método `.unique()` de pandas se identifican y extraen los valores únicos presentes en las columnas driver_fatigue,vehicle_condition, accident_severity, road_type, weather_conditions y driver_gender.

In [13]:
print("\nValores únicos en la columna 'driver_fatigue':")
print(df["driver_fatigue"].unique())

print("\nValores únicos en la columna 'vehicle_condition':")
print(df["vehicle_condition"].unique())

print("\nValores únicos en la columna 'accident_severity':")
print(df["accident_severity"].unique())

print("\nValores únicos en la columna 'road_type':")
print(df["road_type"].unique())

print("\nValores únicos en la columna 'weather_conditions':")
print(df["weather_conditions"].unique())

print("\nValores únicos en la columna 'driver_gender':")
print(df["driver_gender"].unique())

Si hubiera variaciones en la forma en que se representan los datos (por ejemplo, "Male", "male", "M"), sería necesario estandarizar los valores. En este caso no hay variaciones en la forma en que se representan los datos. Sin embargo, la columna "driver_fatigue" presenta valores binarios (0,1) por lo que es óptimo convertirla a dtype booleano usando `.astype(bool)`.

In [19]:
df["driver_fatigue"] = df["driver_fatigue"].astype(bool)

print(df["driver_fatigue"].head())

### Conexión a la base de datos

### Eliminación de columnas redundantes

In [5]:
print("Valores únicos en la columna 'country':")
print(df["country"].unique())

print("\nValores únicos en la columna 'region':")
print(df["region"].unique())

In [19]:
df["driver_fatigue"] = df["driver_fatigue"].astype(bool)

print(df["driver_fatigue"].head())

0    False
1     True
2    False
3     True
4     True
Name: driver_fatigue, dtype: bool


#### Formato de los enteros

La función `verificar_enteros(df)` toma un DataFrame de Pandas como entrada y verifica si todas las columnas de tipo entero contienen únicamente valores enteros válidos. Primero, identifica las columnas de tipo entero y luego itera sobre ellas, comprobando si cada columna es realmente de tipo entero y si todos sus valores son instancias de la clase int o np.integer. Si alguna columna no cumple con estas condiciones, la función imprime un mensaje de error y devuelve "False"; de lo contrario, devuelve True, indicando que todas las columnas enteras contienen solo valores enteros válidos

In [14]:
def verificar_enteros(df):

    columnas_int = df.select_dtypes(include=['int']).columns

    for col in columnas_int:
        if not pd.api.types.is_integer_dtype(df[col]):
            print(f"La columna '{col}' no contiene solo valores enteros.")
            return False

        if not all(isinstance(x, (int, np.integer)) for x in df[col]):
            print(f"La columna '{col}' contiene valores que no son enteros válidos.")
            return False

    return True


verificar_enteros(df)

Para las columnas numéricas no hay valores introducidos erróneamente en otro formato.